# **01 – Exploratory Data Analysis (EDA) – ERA5 Precipitation**

Este notebook tiene por objetivo realizar una exploración inicial de los datos ERA5 de precipitación,
verificar su estructura espacial y temporal, y preparar la base para su uso posterior en el modelo AE+DMD.

---

## **1. Cargar librerías y funciones**

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.data_prep import load_era5_dataset, to_dataframe

## **2. Cargar dataset NetCDF**

In [ ]:
path = "data/raw/era5_precipitation_sample.nc"
ds = load_era5_dataset(path, variable="tp")
ds

## **3. Visualizar estructura temporal**

In [ ]:
ds["tp"].isel(latitude=100, longitude=100).plot()
plt.title("Serie temporal de precipitación (punto específico)")
plt.show()


## **4. Conversión a DataFrame**

In [ ]:
df = to_dataframe(ds, "tp")
df.head()


## **5. Distribución temporal**

In [ ]:
df["date"] = pd.to_datetime(df["time"]).dt.date
daily = df.groupby("date")["value"].mean().reset_index()

plt.figure(figsize=(12,4))
plt.plot(daily["date"], daily["value"], color="royalblue")
plt.title("Precipitación diaria promedio – Chile")
plt.xlabel("Fecha")
plt.ylabel("Precipitación (m)")
plt.grid(True)
plt.show()


## 6. **Mapa promedio espacial**

In [ ]:
mean_map = df.groupby(["latitude", "longitude"])["value"].mean().reset_index()
pivot = mean_map.pivot_table(index="latitude", columns="longitude", values="value")

plt.figure(figsize=(8,6))
plt.imshow(pivot, origin="lower", cmap="Blues", extent=[df.longitude.min(), df.longitude.max(), df.latitude.min(), df.latitude.max()])
plt.title("Precipitación promedio (mm)")
plt.colorbar(label="mm")
plt.xlabel("Longitud")
plt.ylabel("Latitud")
plt.show()


## 7. **Guardar versión procesada**

In [ ]:
from src.data_prep import aggregate_daily, save_processed

df_daily = aggregate_daily(df)
save_processed(df_daily, "data/processed/era5_daily.csv")


📘 Notas:

Sustituir "tp" por la variable que corresponda a precipitación acumulada (depende de la versión de ERA5).

Este EDA puede ampliarse para correlacionar precipitación con temperatura o NDVI satelital.